In [ ]:
# import packages
# set matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from config import dog_vs_cats_config as config
from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import SimplePreprocessor
from pyimagesearch.preprocessing import PatchPreProcessor
from pyimagesearch.preprocessing import MeanPreProcessor
from pyimagesearch.callbacks import TrainingMonitor
from pyimagesearch.io import HDF5DataSetGenerator
from pyimagesearch.nn.conv import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [ ]:
# construct the training image generator for data aug
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
horizontal_flip=True, fill_mode="nearest")

In [ ]:
# load the RGB means for the training set
means = json.loads(open(config.DATASET_MEAN).read())

# init the image preprocess
sp = SimplePreprocessor(227,227)
pp = PatchPreProcessor(227, 227)
mp = MeanPreProcessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# init the training and val dataset generators
trainGen = HDF5DataSetGenerator(config.TRAIN_HDF5, 128, aug=aug,
preprocessors=[pp,mp,iap], classes=2)
valGen = HDF5DataSetGenerator(config.VAL_HDF5, 128,
preprocessors=[pp,mp,iap], classes=2)

In [ ]:
# init the optimizers
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy", optimizers=opt, metrics=["accuracy"])

# construct the set of call backs
path = os.path.sep.join([config.OUTPUT_PATH, "{}.png".format(os.getpid())])
callbacks = [TrainingMonitor(path)]


In [ ]:
# train the network
model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages // 128,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages // 128,
    epochs=75,
    max_queue_size=10,
    callbacks=callbacks, verbose=1)

In [ ]:
# save the model to file
print("[INFO] serializing model...")
model.save(config.MODEL_PATH, overwrite=True)

# close the HDF5 datasets
trainGen.close()
valGen.close()